<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #15: Process Spyfish Aotearoa videos</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Sep 15th, 2021</h5>

# Set up and requirements

Import Python packages

In [ ]:
!pip install boto3

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Import required modules
import utils.t15_utils as t15
import utils.t12_utils as t12
from utils.zooniverse_utils import retrieve_zoo_info, populate_subjects

print("Packages loaded successfully")

In [ ]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Import required modules
import utils.t15_utils as t15
import utils.t12_utils as t12

print("Packages loaded successfully")

import os
import pandas as pd
import sqlite3
import pandas as pd
from tqdm import tqdm
import subprocess

import utils.db_utils as db_utils
import utils.server_utils as server_utils
import utils.spyfish_utils as spyfish_utils

# Get the location of the csv files with initial info to populate the db
sites_csv, movies_csv, species_csv = server_utils.get_sites_movies_species()

# Load the csv with movies information
movies_df = pd.read_csv(movies_csv)

# Select only those deployments that are valid
movies_df = movies_df[~movies_df["IsBadDeployment"]].reset_index(drop=True)    

In [2]:
aws_access_key_id, aws_secret_access_key = server_utils.aws_credentials()

Enter the key id for the aws server········
Enter the secret access key for the aws server········


In [3]:
# Check that all videos have filenames
if movies_df["filename"].isna().any():
    
    #####Get info from bucket#####
    # Your acess key for the s3 bucket. 
    #aws_access_key_id, aws_secret_access_key = server_utils.aws_credentials()

    # Specify the bucket where the BUV files are
    bucket_i = movies_df['bucket'].str.split('/').str[0].dropna().unique()[1]

In [4]:
client = server_utils.connect_s3(aws_access_key_id, aws_secret_access_key)

In [5]:
# Select only surveys that are missing filenames
unprocessed_movies_df = movies_df[movies_df["filename"].isna()].reset_index(drop=True)

# Write the filename of the concatenated movie
unprocessed_movies_df["filename"] = unprocessed_movies_df["siteName"] + "_" + unprocessed_movies_df["created_on"].str.replace('/','_')+ ".MP4"

In [6]:
unprocessed_movies_df

,movie_id,filename,siteName,created_on,Author,fps,duration,survey_start,survey_end,go_pro_files,bucket,IsBadDeployment
0,35,TUH_008_22_09_2020.MP4,TUH_008,22/09/2020,MoniqueLadds,99,7200,104,1904,Site 8 video 1.MP4;Site 8 video 2.MP4;Site 8 v...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
1,36,TUH_039_22_09_2020.MP4,TUH_039,22/09/2020,MoniqueLadds,99,7200,60,1860,Site 39 video 1.MP4;Site 39 video 2.MP4;Site 3...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
2,37,TUH_040_22_09_2020.MP4,TUH_040,22/09/2020,MoniqueLadds,99,7200,264,2064,Site 40 video 1.MP4;Site 40 video 2.MP4;Site 4...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
3,38,TUH_041_22_09_2020.MP4,TUH_041,22/09/2020,MoniqueLadds,99,7200,360,2160,Site 41 video 1.MP4;Site 41 video 2.MP4;Site 4...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
4,39,TUH_035_22_09_2020.MP4,TUH_035,22/09/2020,MoniqueLadds,99,7200,95,1895,Site 35 video 1.MP4;Site 35 video 2.MP4;Site 3...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
5,41,TUH_007_31_08_2020.MP4,TUH_007,31/08/2020,MoniqueLadds,99,7200,682,2482,Site 7 video 1.MP4;Site 7 video 2.MP4;Site 7 v...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
6,42,TUH_003_31_08_2020.MP4,TUH_003,31/08/2020,MoniqueLadds,99,7200,682,2482,Site 03 video 1.MP4;Site 03 video 2.MP4;Site 0...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
7,43,TUH_034_31_08_2020.MP4,TUH_034,31/08/2020,MoniqueLadds,99,7200,380,2180,Site 034 video 1.MP4;Site 034 video 2.MP4;Site...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
8,44,TUH_004_31_08_2020.MP4,TUH_004,31/08/2020,MoniqueLadds,99,7200,80,1880,Site 04 video 1.MP4;Site 04 video 2.MP4;Site 0...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False
9,49,TUH_037_1_09_2020.MP4,TUH_037,1/09/2020,MoniqueLadds,99,7200,395,2195,Site 37 video 1.MP4;Site 37 video 2.MP4;Site 3...,marine-buv/buv-zooniverse-uploads/tuhua-buv-20...,False


In [ ]:
concatenate_videos(unprocessed_movies_df, client)

  0%|          | 0/3 [00:00<?, ?it/s]

Site 8 video 1.MP4: 100%|██████████| 4.00G/4.00G [05:49<00:00, 11.5MB/s]

Site 8 video 2.MP4:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

Site 8 video 2.MP4: 100%|██████████| 4.00G/4.00G [05:36<00:00, 11.9MB/s]

Site 8 video 3.MP4:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

Site 8 video 3.MP4: 100%|██████████| 4.00G/4.00G [05:12<00:00, 12.8MB/s]

100%|██████████| 3/3 [16:39<00:00, 333.13s/it]


Concatenating  TUH_008_22_09_2020.MP4


TUH_008_22_09_2020.MP4:   0%|          | 0.00/12.0G [00:00<?, ?B/s]

TUH_008_22_09_2020.MP4 concatenated successfully


TUH_008_22_09_2020.MP4:  67%|██████▋   | 8.06G/12.0G [1:00:08<24:23, 2.69MB/s]  

In [ ]:
unprocessed_movies_df

In [ ]:
# Retrieve info from the bucket
contents_s3_pd = server_utils.retrieve_s3_buckets_info(client, bucket_i)

In [ ]:
contents_s3_pd

In [ ]:
contents_s3_pd[contents_s3_pd['Key'].str.contains("buv-zooniverse-uploads/tuhua-buv-2020/TUH_021/TUH_021__22_09_2020.MP4")]['Key'].unique()

In [ ]:
#client.delete_object(Bucket=bucket_i, Key="buv-zooniverse-uploads/tuhua-buv-2020/TUH_021/TUH_021__22_09_2020.MP4")

In [ ]:
# Specify the filename of the raw videos        
contents_s3_pd['raw_filename'] = contents_s3_pd['Key'].str.split('/').str[-1]

# Specify the filename of the raw videos        
contents_s3_pd['bucket'] = contents_s3_pd['Key'].str.rsplit('/',1).str[0]

In [ ]:
session = server_utils.connect_s3(aws_access_key_id, aws_secret_access_key)